<a href="https://colab.research.google.com/github/iremellz/Mortgage_Data_EDA/blob/main/Mortgage%20Data%20EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading the CSV File and Column Data to Create Dataframe for Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from pandas.core.algorithms import mode
import random

## Uploading the CSV file to Python

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!gdown 1B1510RsBuaaAWYxXR1f966LVGh2OAuk-

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1B1510RsBuaaAWYxXR1f966LVGh2OAuk- 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Uploading Column Info Provided in Appendix A to Python for Processing the CSV File

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
fileObject = open("column_names_types.txt", "r")
columns = fileObject.read()


In [ ]:
column_info=columns.split(',')
column_info

In [ ]:
column_name= column_info[::2]

In [ ]:
column_name= column_name[0:110]

In [ ]:
column_name

In [ ]:
column_type= column_info[1::2]

In [ ]:
column_type

In [ ]:
col_dict= dict(zip(column_name,column_type))
print(col_dict)

## Creating the Dataframe and Initial Analysis/Cleaning of Dataframe

In [ ]:
df= pd.read_csv('Group_Project_1.csv', sep= '|', header=None, names=column_name, dtype=col_dict)

In [ ]:
df

In [ ]:
performance_vars=['act_period','loan_age','rem_months','dlq_status','zero_bal_code','zb_dte',]

In [ ]:
acqusition_vars=[
'loan_id',
'channel',
'orig_rate',
'orig_upb',
'orig_term',
'orig_date',
'first_pay',
'matr_dt',
'oltv',
'ocltv',
'num_bo',
'dti',
'cscore_b',
'cscore_c',
'first_flag',
'purpose',
'prop',
'no_units',
'occ_stat',
'state',
'msa',
'zip',
'mi_pct',
'product',
'ppmt_flg',
'io',
'first_pay_io',
'mi_type',
'homeready_program_indicator',
'relocation_mortgage_indicator',
'high_balance_loan_indicator',
'arm_5_yr_indicator',
'arm_product_type',
'months_until_first_payment_reset',
'months_between_subsequent_payment_reset',
'arm_index',
'arm_cap_structure',
'initial_interest_rate_cap',
'periodic_interest_rate_cap',
'lifetime_interest_rate_cap',
'margin',
'balloon_indicator',
'plan_number',
'high_loan_to_value_hltv_refinance_option_indicator',
]

In [ ]:
useful_vars= performance_vars+acqusition_vars

In [ ]:
parse_dates=['orig_date','first_pay','zb_dte','act_period','matr_dt']
for i in parse_dates:
  df[i]=pd.to_datetime(df[i])

In [ ]:
print(df.dtypes)

In [ ]:
df= df[useful_vars]

In [ ]:
df.shape

In [ ]:
df.describe().round(2)

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum())

# Downsampling and Creating Termination States

##Downsampling

In [ ]:
df_sample=df.sample(frac =0.1, axis=0, random_state=0)
df_sample

In [ ]:
df_num = df_sample._get_numeric_data()
df_num.describe().round(2)


## Creating Termination States

In [ ]:
def loan_status (row):
  if row['zero_bal_code'] in ['09','03','02','06','15','16']:
    return "D"
  elif row['zero_bal_code'] in ['01']:
    return "P"
  elif row ['dlq_status'] not in ('00','01','02','03','04','05'):
    return "D"
  else:
    return "A"

df_sample["target"] = df_sample.apply(loan_status, axis=1)

In [ ]:
df_sample.groupby(['target'], dropna=False)['loan_id'].count()

# Cleaning the Rows

In [ ]:
df_not_active= df_sample[df_sample['target']!= 'A'].groupby(['loan_id']).agg({'act_period':'min'}).\
reset_index().rename(columns={'act_period':'last_period'})

In [ ]:
df_not_active.keys()

In [ ]:
df_sample= df_sample.merge(df_not_active, on='loan_id', how='left')

In [ ]:
df_sample.target.value_counts()

In [ ]:
df_sample = df_sample.loc[~((df_sample['act_period'] > df_sample['last_period'])),:]

In [ ]:
df_sample.target.value_counts()

## Analysing the Dataframe for Missing Values and Dropping Columns with Excessive Number of Missing Values

In [ ]:
obs_count = df_sample.shape[0]
obs_count

In [ ]:
missings =(df_sample.isnull().sum()/obs_count).sort_values(ascending=False).to_frame()
missings


In [ ]:
missing_cols= ['margin',
'lifetime_interest_rate_cap',
'months_between_subsequent_payment_reset',
'months_until_first_payment_reset',
'arm_product_type',
'arm_cap_structure',
'arm_5_yr_indicator',
'initial_interest_rate_cap',
'periodic_interest_rate_cap',
'arm_index',
'balloon_indicator',
'plan_number',
'first_pay_io']

In [ ]:
df_sample.drop(missing_cols, axis=1, inplace=True)

In [ ]:
df_sample.columns

In [ ]:
df_sample

## Creating Additional Columns for Data Analysis

In [ ]:
df_sample['act_period'] = pd.to_datetime(df_sample['act_period'])
df_sample['act_period'].day = 30
df_sample['calc_age']= ((df_sample['act_period']-df_sample['orig_date'])/np.timedelta64(1,'M'))
df_sample['calc_age'] = df_sample['calc_age'].astype(int)+1

In [ ]:
df_sample.loc[(df_sample['loan_age'].isnull()), "loan_age"] = df_sample['calc_age']

In [ ]:
df_sample['age_dif'] = df_sample['calc_age']-df_sample['loan_age']

In [ ]:
print(df_sample.columns)

In [ ]:
df_sample['calc_rem_months'] = df_sample['orig_term']- df_sample['loan_age']

In [ ]:
df_sample.shape

In [ ]:
df_sample['calc_rem_months'] = df_sample['calc_rem_months'].astype(int)
df_sample.loc[(df_sample['rem_months'].isnull()), "rem_months"] = df_sample['calc_rem_months']
missings=df_sample['rem_months'].isnull().sum()
print(missings)

In [ ]:
df_sample.isna().sum()

## Imputing cscore_b with cscore_c or vice versa

In [ ]:
len(df_sample.loc[(df_sample['cscore_b'] == 'NaN')])

In [ ]:
df_sample['cscore_b'] = np.where(df_sample['cscore_b'] == 'NaN', df_sample['cscore_c'], df_sample['cscore_b'])

In [ ]:
df_sample['cscore_c'] = np.where(df_sample['cscore_c'] == 'NaN', df_sample['cscore_b'], df_sample['cscore_c'])

## Imputing ocltv with oltv

In [ ]:
df_sample['ocltv'] = np.where(df_sample['ocltv'] == 'NaN', df_sample['oltv'], df_sample['ocltv'])

## Imputing the missing values in numerical and categorical columns

In [ ]:
def fill_numerical_values(df, x_var):
  df[x_var] = df[x_var].fillna(df[x_var].median())

In [ ]:
def fill_cat_values (df, x_var):
  df[x_var] = df[x_var].fillna(df[x_var].mode())

In [ ]:
#def fill_categorical_values (df, x_var):
  #df = df.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [ ]:
df_sample.channel.mode()

In [ ]:
imp_num = SimpleImputer(strategy='mean') #imputing numerical columns with the mean

In [ ]:
imp_cat= SimpleImputer(strategy='most_frequent') #imputing categorical columns with the mode

In [ ]:
df_sample.columns

In [ ]:
num_cols = ['orig_rate', 'orig_upb', 'orig_term', 'loan_age', 'rem_months', 'oltv', 'ocltv', 'dti', 'cscore_b', 'cscore_c', 'mi_pct', 'calc_rem_months', 'age_dif', 'calc_age' ]

In [ ]:
cat_columns= ['channel', 'first_flag', 'purpose', 'prop', 'no_units', 'occ_stat', 'state', 'product', 'ppmt_flg', 'io', 'mi_type', 'homeready_program_indicator', 'relocation_mortgage_indicator', 'high_balance_loan_indicator', 'high_loan_to_value_hltv_refinance_option_indicator', 'num_bo']

In [ ]:
for i in num_cols:
  fill_numerical_values(df_sample, i)

In [ ]:
for i in cat_columns:
  fill_cat_values(df_sample, i)

In [ ]:
for i in cat_columns:
  df_sample[i].fillna(df_sample[i].mode().iloc[0], inplace=True)
#Imputing categorical columns with the most common element in the column

In [ ]:
df_sample.isna().sum()

In [ ]:
# df_sample.drop(['zb_dte'], inplace= True, axis=1)
# zb_dte has the same number of missing columns as zero_bal_code and we believe that there is a strong correlation between these columns
# therefore we dropped zb_dte column.

In [ ]:
from locale import normalize
df_sample['mi_type'].value_counts(normalize=True)

In [ ]:
# we tried to write a code to impute the mi_type column with the current distribution of the existing values. However, there was an issues due to random variables and
# we skipped this part in order to follow up the basic instructions that told us to impute the value with the most frequent value of this column.


# from pandas.core.common import random_state
#import random
# for i in range(0,3):
#  j=0
#  while j <(len(df_sample)*df_sample['mi_type'].value_counts(normalize=True)[i]-1):
#    if df_sample[random.randint(0, 557672)]['mi_type'].isna()==True or df_sample[random.randint(0, 557672)]['mi_type']==0:
#      df_sample[random.randint(0, 557672)]['mi_type']=i
#      j +=1

In [ ]:
df_sample.isna().sum()

In [ ]:
obs_count = df_sample.shape[0]
obs_count
missings =(df_sample.isnull().sum()/obs_count).sort_values(ascending=False).to_frame()
missings

In [ ]:
imp_cat.fit_transform(df_sample)

In [ ]:
obs_count = df_sample.shape[0]
obs_count
missings =(df_sample.isnull().sum()/obs_count).sort_values(ascending=False).to_frame()
missings

# Creating New Variables

## Creating 'origination_value' Variable

In [ ]:
df_sample['orig_upb']

In [ ]:
df_sample['oltv']

In [ ]:
#converting 'oltv' to decimal

df_sample['oltv_d']=df_sample['oltv'].astype(float)/100

In [ ]:
df_sample['orig_value']= (df_sample['orig_upb']/df_sample['oltv_d'])

In [ ]:
df_sample['orig_value']

## Creating 'vintage' Variable

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_sample['Vintage'] = df_sample['orig_date'].dt.year
plt.hist(df_sample['Vintage']);

In [ ]:
cat_columns= cat_columns + ['Vintage']

In [ ]:
num_cols = num_cols + ['orig_value']

## Checking correlation among variables

In [ ]:
df_sample.corr()
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df_sample.corr(), annot= True);

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df_sample.corr(method='kendall'), annot= True);

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df_sample.corr(method='spearman'), annot= True);

# Checking Outliers

In [ ]:

#np.random.seed(1234)

#plt.subplot(2, 1, 1)
#df_sample = pd.DataFrame(np.random.randn(15, 11),
                   #columns=['orig_rate', 'orig_upb', 'orig_term', 'loan_age', 'rem_months', 'oltv', 'ocltv', 'dti', 'cscore_b', 'cscore_c', 'mi_pct'])
#df_sample.boxplot(column=['orig_rate', 'orig_upb', 'orig_term', 'loan_age', 'rem_months', 'oltv', 'ocltv', 'dti', 'cscore_b', 'cscore_c', 'mi_pct'], grid= False, fontsize = 15, rot = 60, figsize=(9,9))

#plt.subplot(2, 1, 2)

#df_sample(column=num_cols, grid= False, fontsize = 15, rot = 60, figsize=(9,9))


#fig, axs = plt.subplots(5, 2, figsize=(10,10), sharey=False)

#df_sample.boxplot('orig_rate', ax=ax[0,0])
#df_sample.boxplot('orig_upb', ax=ax[0,1])
#df_sample.boxplot('orig_term', ax=ax[0,2])
#df_sample.boxplot('loan_age', ax=ax[0,3])
#df_sample.boxplot('rem_months', ax=ax[0,4])

#fig, axs = plt.subplots(1, 6, figsize=(10,10), sharey=False)

#df_sample.boxplot('oltv', ax=ax[0,5])
#df_sample.boxplot('ocltv', ax=ax[0,6])
#df_sample.boxplot('dti', ax=ax[0,7])
#df_sample.boxplot('cscore_b', ax=ax[0,8])
#df_sample.boxplot('cscore_c', ax=ax[0,9])
#df_sample.boxplot('mi_pct', ax=ax[0,10])

#plt.subplots_adjust(wspace=0.5)

#plt.show()

In [ ]:
#plt.figure(figsize=(20,5))
#for i in range(1, len(df_sample.columns)+1):
  #plt.subplot(1, len(df_sample.columns), i)
  #sns.boxplot(data=df_sample.iloc[:, i-1] )

In [ ]:
plt.figure(figsize=(30,20))

num_cols1 = ['orig_rate', 'orig_upb', 'orig_term', 'calc_age', 'calc_rem_months', 'age_dif']
fig, axes = plt.subplots(1, len(num_cols1))
for i, col in enumerate(num_cols1):
    ax = sns.boxplot(y=df_sample[col], ax=axes.flatten()[i])

plt.subplots_adjust(wspace=2);



num_cols2 = ['loan_age', 'rem_months', 'oltv', 'ocltv', 'orig_value']
fig, axes = plt.subplots(1, len(num_cols2))
for i, col in enumerate(num_cols2):
    ax = sns.boxplot(y=df_sample[col], ax=axes.flatten()[i])

plt.subplots_adjust(wspace=1.5);



num_cols3 = ['dti', 'cscore_b', 'cscore_c', 'mi_pct']
fig, axes = plt.subplots(1, len(num_cols3))
for i, col in enumerate(num_cols3):
    ax = sns.boxplot(y=df_sample[col], ax=axes.flatten()[i])

plt.subplots_adjust(wspace=1.5);


plt.show()

In [ ]:
# !!!!!!

def cap_floor(df, x_val):
  q_l = df[x_val].quantile(0.01)
  q_h = df[x_val].quantile(0.99)

  df.loc[df[x_val]<= q_l, x_val] = q_l
  df.loc[df[x_val]>= q_h, x_val] = q_h


  #print(q_l)
  #print(q_h)

  # df_filtered = df[(df[x_val] > q_l) & (df[x_val] < q_h)] (you can use or)

  # return df_filtered


  #df = df.drop('q_l', axis=0)
  #df = df.drop('q_h', axis=0)


In [ ]:
df_sample.shape

In [ ]:
for i in (num_cols):

  cap_floor(df_sample, i)

# Variable Relationship with the target

In [ ]:
df_sample['Default']= df_sample.apply(lambda x : 1 if x['target']=='D' else 0,axis=1)

In [ ]:
def univariate_plot_v3(df, x_var, y_var, bin_size):
  if x_var in num_cols:
    fig,ax = plt.subplots()
    s=df.groupby([pd.cut(df[x_var], bins=bin_size, precision =0)]).agg({'loan_id':'count', y_var:'mean'})
    sns.barplot(x=x_var, y='loan_id', data=s.reset_index(), palette="Blues_d", ax=ax)
    ax2 = ax.twinx()
    sns.lineplot(x=range(len(s.reset_index())), y=y_var, data=s.reset_index(), color='red', markers=True, ax=ax2)
    ax.set_xticklabels(s.index.values, rotation = 45, ha="right")

  elif x_var in cat_cols:
    fig,ax = plt.subplots()
    s=df.groupby(df[x_var]).agg({'loan_id':'count', y_var:'mean'})
    sns.barplot(x=x_var, y='loan_id', data=s.reset_index(), palette="Blues_d", ax=ax)
    ax2 = ax.twinx()
    sns.lineplot(x=range(len(s.reset_index())), y=y_var, data=s.reset_index(), color='red', markers=True, ax=ax2)
    ax.set_xticklabels(s.index.values, rotation = 45, ha="right")


In [ ]:
cat_cols= ['channel', 'first_flag', 'purpose', 'prop', 'no_units', 'occ_stat', 'state', 'product', 'ppmt_flg', 'io', 'mi_type', 'homeready_program_indicator', 'relocation_mortgage_indicator', 'high_balance_loan_indicator', 'high_loan_to_value_hltv_refinance_option_indicator', 'num_bo', 'Vintage']

In [ ]:
vars_to_plot = cat_columns + num_cols

for i in (vars_to_plot):
  univariate_plot_v3(df_sample,i,'Default', 10)

In [ ]:
# dropping the columns that has no power
cols_no_variation = ['product', 'ppmt_flg','high_loan_to_value_hltv_refinance_option_indicator', 'io']

In [ ]:
df_sample.drop(cols_no_variation, inplace=True, axis=1)

In [ ]:
vars_to_plot = [x for x in vars_to_plot if x not in cols_no_variation]

In [ ]:
for i in (vars_to_plot):

  univariate_plot_v3(df_sample,i,'Default', 10)

# Variable Relationships with the competing terminal state

In [ ]:
def univariate_plot_overlay(df, x_var, y_var_1, y_var_2, bin_size):
  if x_var in num_cols:
    s1=df.groupby([pd.cut(df[x_var], bins=10)]).mean([y_var_1,y_var_2])
    p1=sns.lineplot(data=s1, x=s1[x_var], y=y_var_1, label=y_var_1)
    p2=sns.lineplot(data=s1, x=s1[x_var], y=y_var_2, label=y_var_2)

  elif x_var in cat_cols:
    s1=df.groupby(df[x_var]).mean([y_var_1,y_var_2])

    fig,ax = plt.subplots()

    s1=s1[[y_var_1, y_var_2]]
    s1[x_var]=s1.index
    print(s1)
    s1.plot.bar(x=x_var, ax=ax)

    plt.xticks(rotation=45)
    plt.show();


In [ ]:
df_sample['Prepay']=df_sample.apply(lambda row : 1 if row['target']=='P' else 0, axis=1)

In [ ]:
for i in (vars_to_plot):

  univariate_plot_overlay(df_sample, i, 'Default', 'Prepay', 10)

# Conclusion

*1*. If the mortgage channel is ‘Broker’, then there is a higher probability of ‘Default’.
1. First time home buyers are more likely to default compared to those who are    getting their second or other types of mortgages.
1. If borrower access the cash value inside the property it is more likely to default.
1. If the house is manufactured house there is high likelihood to default.

1. If it is an investment or rental property there is a more likelihood default. Those people who are living in their primary residence are less likelihood to default.

1. If the borrower is invested there is a high probability of default.

1. VI has highest default, FL has second highest and RP has third highest default ratio.

1. If the lender paid the mortgage insurance premium payment there is higher probability of default.

1. If the borrower participate the HomeReady program there is high default ratio.  

1. If a loan is given to a person who is relocating because of work the default ratio is smaller. It can also mean that employment status has a negative correlation with the default rate.

1. Those borrowers who purchase expensive properties with large mortgage are less likely to default.

1. When there was an economic downturn (2006-2008) there were high defaults.

1. When the mortgage rate increases, there is a higher probability of default.

1. When the loan size is increasing after $400k, the default value is decreasing. It can mean prosperous borrowers who purchased more expensive properties are less likely to default.

1. If the length of the term of the loan is longer, the default rate increases. The smaller the term, the less likely borrow to default.

1. If debt-to-income ratio increases, there is a high likelihood of default.

1. If the borrower has higher credit score it is less likely to default.

1. If remanning months between 20-25 years it has the highest probability of default.